In [1]:
from __future__ import print_function
from __future__ import absolute_import
from __future__ import division
from keras.models import Model
from keras.layers import *
from keras.regularizers import l2
from keras.utils import conv_utils
from keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.applications.resnet50 import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras import backend as K

import numpy as np 
import pandas as pd

from keras.utils import to_categorical
data_train = pd.read_csv('~/fashion-mnist_train.csv')
data_test = pd.read_csv('~/fashion-mnist_test.csv')

X_train = np.array(data_train.iloc[:, 1:])
X_test = np.array(data_test.iloc[:, 1:])
y_train = to_categorical(np.array(data_train.iloc[:, 0]))
y_test = to_categorical(np.array(data_test.iloc[:, 0]))

img_rows, img_cols = 28, 28
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

def squeeze_excite_block(input, ratio=16):
    
    init = input
    filters = init._keras_shape[-1]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', use_bias=False)(se)
    x = multiply([init, se])
    
    return x

def SEResNet(input_shape = (img_rows, img_cols, 1),
             initial_conv_filters=64,
             depth=[2, 2, 2, 2],
             filters=[64, 128, 256, 512],
             width=1,
             weight_decay=1e-4,
             include_top=True,
             weights=None,
             input_tensor=None,
             pooling=None,
             classes=10): 

    img_input = Input(shape=input_shape)
   
    x = _create_se_resnet(classes, img_input, include_top, initial_conv_filters,
                          filters, depth, width, weight_decay, pooling)


    model = Model(img_input, x, name='resnext')
    print('model created')
    
    model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])
    model.summary()
    
    model.fit(X_train, y_train,
          batch_size=50,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test))
    
    score = model.evaluate(X_test, y_test, verbose=0)
    
    print('accuracy: ',score[1])
    print('loss: ',score[0])
    return model


def _resnet_block(input, filters, k=1, strides=(1, 1)):
    init = input
    channel_axis = -1 

    x = BatchNormalization(axis=channel_axis)(input)
    x = Activation('relu')(x)

    if strides != (1, 1) or init._keras_shape[channel_axis] != filters * k:
        init = Conv2D(filters * k, (1, 1), padding='same', kernel_initializer='he_normal',
                      use_bias=False, strides=strides)(x)

    x = Conv2D(filters * k, (3, 3), padding='same', kernel_initializer='he_normal',
               use_bias=False, strides=strides)(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)

    x = Conv2D(filters * k, (3, 3), padding='same', kernel_initializer='he_normal',
               use_bias=False)(x)

    # squeeze and excite block
    x = squeeze_excite_block(x)

    m = add([x, init])
    return m




def _create_se_resnet(classes, img_input, include_top, initial_conv_filters, filters,
                      depth, width, weight_decay, pooling):

    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    N = list(depth)

    x = Conv2D(initial_conv_filters, (7, 7), padding='same', use_bias=False, strides=(2, 2),
               kernel_initializer='he_normal', kernel_regularizer=l2(weight_decay))(img_input)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    for i in range(N[0]):
            x = _resnet_block(x, filters[0], width)

    for k in range(1, len(N)):
            x = _resnet_block(x, filters[k], width, strides=(2, 2))

    for i in range(N[k] - 1):
            x = _resnet_block(x, filters[k], width)

    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)


    x = GlobalAveragePooling2D()(x)
    x = Dense(classes, use_bias=False, kernel_regularizer=l2(weight_decay),
                  activation='softmax')(x)
    
    return x

if __name__ == "__main__":
    SEResNet()

Using TensorFlow backend.


model created
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 28, 28, 1)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 14, 14, 64)    3136        input_1[0][0]                    
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 7, 7, 64)      0           conv2d_1[0][0]                   
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 7, 7, 64)      256         max_pooling2d_1[0][0]            
_____________________________________________________________________________

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 99s - loss: 0.4255 - acc: 0.8500 - val_loss: 0.3596 - val_acc: 0.8708
Epoch 2/10
60000/60000 [==============================] - 84s - loss: 0.3074 - acc: 0.8908 - val_loss: 0.3417 - val_acc: 0.8780
Epoch 3/10
60000/60000 [==============================] - 89s - loss: 0.2667 - acc: 0.9052 - val_loss: 0.2929 - val_acc: 0.8937
Epoch 4/10
60000/60000 [==============================] - 100s - loss: 0.2421 - acc: 0.9148 - val_loss: 0.3444 - val_acc: 0.8751
Epoch 5/10
60000/60000 [==============================] - 83s - loss: 0.2184 - acc: 0.9228 - val_loss: 0.2695 - val_acc: 0.9020
Epoch 6/10
60000/60000 [==============================] - 91s - loss: 0.1997 - acc: 0.9307 - val_loss: 0.2687 - val_acc: 0.9063
Epoch 7/10
60000/60000 [==============================] - 97s - loss: 0.1796 - acc: 0.9370 - val_loss: 0.2606 - val_acc: 0.9097
Epoch 8/10
60000/60000 [=============================

In [2]:

import keras
keras.__version__

'2.0.8'